Chicago Crime Plot Comparison

In [34]:
!pip install bokeh

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.


In [61]:
## Main Chicago Crime Plot

import pandas as pd
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, HoverTool, Range1d, LinearAxis
from bokeh.models.tickers import YearsTicker

# Read in the Crime Data csv file
#df = pd.read_csv('/data/workspace_files/DailyViolentCrimesv1.csv',parse_dates=['CrimeDate'],dayfirst=True)
df = pd.read_csv('https://gitlab.com/ds-team-1/chicagocrimeforecaster/-/raw/main/data/DailyViolentCrimesv1.csv',parse_dates=['CrimeDate'],dayfirst=True)
df.sort_values('CrimeDate', inplace=True)

# Output to static HTML file
output_file('CHGCrime.html',
            title='Chicago Crime Chart')
output_notebook()

# Create a ColumnDataSource
crime_cds = ColumnDataSource(df)

# Create view for each component (two graphs)
crimecount_view = CDSView(source=crime_cds)
feels_view = CDSView(source=crime_cds)
# add headroom for graph
y_overlimit = 0.08 # show y axis below and above y min and max value
#p = figure()

# Create and configure the figure
# Make the grid background, size to 16:9 format to fit presentation
crime_fig = figure(x_axis_type='datetime',
                   plot_height=768, plot_width=1377,
                   title='Violent Crime Influence Comparison',
                   x_axis_label='Date', y_axis_label='Violent Crime Rate',
                   toolbar_location="above",
                   y_range = (df['Crime_Count'].min() * (1 - y_overlimit),
                              220))
#                              df['Crime_Count'].max() * (1 + y_overlimit)))

#crime_fig.x_range.min_interval = 2010
#crime_fig.x_range.max_interval = 2020
crime_fig.xaxis.ticker.desired_num_ticks = 10
crime_fig.grid.grid_line_color = 'deepskyblue'


# Setting the second y axis range name and range
crime_fig.extra_y_ranges = {'feels_like': Range1d(df['feels_like'].min() * (1 - y_overlimit), 
                                           df['feels_like'].max() * (1 + y_overlimit)),
                            'humidity': Range1d(df['humidity'].min() * (1 - y_overlimit), 
                                           df['humidity'].max() * (1 + y_overlimit)),
                            'precipitation': Range1d(df['precipitation'].min() * (1 - y_overlimit), 
                                           df['precipitation'].max() * (1 + y_overlimit)),
                            'unemployment': Range1d(df['unemployment'].min() * (1 - y_overlimit), 
                                           df['unemployment'].max() * (1 + y_overlimit)),
                            'Adj_Close': Range1d(df['Adj_Close'].min() * (1 - y_overlimit), 
                                           df['Adj_Close'].max() * (1 + y_overlimit)) }

# Adding the second axis to the plot.  
crime_fig.add_layout(LinearAxis(y_range_name='feels_like', axis_label='"Feels Like" Temperature'),'right')

# Format the tooltip
tooltips = [
            ('Crime_Count', '@Crime_Count'),
#            (comparison_field, '@'+comparison_field),
            ]

#crime_fig.add_tools(HoverTool(tooltips=tooltips))

### main plot of Crime_Count as RED
crime_fig.line(x='CrimeDate', y='Crime_Count',
              source=crime_cds, view=crimecount_view,
              color='#CE1141', legend_label='Crime_Count')

### second plot GREEN of feels_like
crime_fig.x('CrimeDate', 'feels_like',
              source=crime_cds, view=crimecount_view,
              color='#39fc03', legend_label='feels_like',
              y_range_name='feels_like',)

### Third plot Orange of humidity
crime_fig.y('CrimeDate', 'humidity',
              source=crime_cds, view=crimecount_view,
              color='#fc8403', legend_label='humidity',
              y_range_name='humidity',)

### Fourth plot Yellow of precipitation
crime_fig.line('CrimeDate', 'precipitation',
              source=crime_cds, view=crimecount_view,
              color='#fce703', legend_label='precipitation',
              y_range_name='precipitation',)

### Fifth plot White of Unemployment
crime_fig.line('CrimeDate', 'unemployment',
              source=crime_cds, view=crimecount_view,
              color='white', legend_label='unemployment',
              y_range_name='unemployment',)

### Sixth plot Blue of Adj_Close
crime_fig.line('CrimeDate', 'Adj_Close',
              source=crime_cds, view=crimecount_view,
              color='mediumblue', legend_label='Adj_Close',
              y_range_name='Adj_Close',)

# Move the legend to the upper left corner
crime_fig.legend.location = 'top_left'
crime_fig.legend.orientation = "horizontal"
crime_fig.legend.border_line_width = 3
crime_fig.legend.border_line_color = "blue"
crime_fig.legend.border_line_alpha = 0.7
crime_fig.legend.title_text_font_size = "14px"
crime_fig.legend.inactive_fill_color = "cornflowerblue"
crime_fig.legend.inactive_fill_alpha = 0.7
crime_fig.legend.background_fill_color = "silver"
crime_fig.legend.title = "Click Parameter Labels To Toggle Visibility"
crime_fig.legend.title_text_font_style = "bold"

crime_fig.title.text_font_size = '18pt'
crime_fig.background_fill_color = "slategray"

# Toggle legend
crime_fig.legend.click_policy="hide"

# Show the plot
show(crime_fig)
